# 接口

TypeScript 用值的**形状（Shape）**来进行类型检查，接口用来描述这些**类型**。

In [1]:
interface LabeledValue {
    label: string
}

function printLabel(labeledObj: LabeledValue) {
    console.log(labeledObj.label)
}

let myObj = {label: 'Size 10 Object', size: 10}

printLabel(myObj)

Size 10 Object


undefined

*注意 `myObj` 含有不在 `LabeledValue` 接口定义内的额外属性*

## 可选属性

In [2]:
interface SquareConfig {
    color?: string
    width: number
}

interface Square {
    color?: string
    area: number
}

function createSquare(config: SquareConfig): Square {
    let newSquare: Square = {
        color: 'white',
        area: config.width ** 2,
    }
    if (config.color) {
        newSquare.color = config.color
    }
    return newSquare
}

let square: Square = createSquare({width: 15})
console.log(square)

{ color: 'white', area: 225 }


undefined

## 只读属性

In [3]:
interface Point {
    readonly x: number
    readonly y: number
}

let point: Point = {
    x: 10,
    y: 10,
}

undefined

In [4]:
point.x = 20

Error: Line 1, Character 7
point.x = 20
______^
TS2540: Cannot assign to 'x' because it is a read-only property.

#### `ReadonlyArray<T>`

`ReadonlyArray<T>` 和 `Array<T>` 一样，但是没有任何修改方法。数组被创建后将不能被修改。

In [5]:
let readonlyArray: ReadonlyArray<number> = [1, 2, 3, 4]

undefined

In [6]:
readonlyArray[0] = 9

Error: Line 1, Character 1
readonlyArray[0] = 9
^
TS2542: Index signature in type 'readonly number[]' only permits reading.

不能将只读数组再转化为普通数组：

In [7]:
let array: number[] = readonlyArray

Error: Line 1, Character 5
let array: number[] = readonlyArray
____^
TS2740: Type 'readonly number[]' is missing the following properties from type 'number[]': pop, push, reverse, shift, and 6 more.

除非进行强制类型断言

In [8]:
let array: number[] = readonlyArray as number[]
console.log(array)

[ 1, 2, 3, 4 ]


undefined

## 额外检查

注意我们前边的第一个接口例子，我们的 `myObj` 有接口定义的额外属性，但是代码正常运行。但是如果我们将**对象字面量**作为参数传递给函数，函数将会报错：

In [9]:
printLabel({
    label: 'Size 10 Object', 
    size: 10,
})

Error: Line 3, Character 5
    size: 10,
____^
TS2345: Argument of type '{ label: string; size: number; }' is not assignable to parameter of type 'LabeledValue'.
  Object literal may only specify known properties, and 'size' does not exist in type 'LabeledValue'.

TypeScript 抱怨说 `LabeledValue` 不包含 `size` 属性。**如果对象字面量包含任何目标类型不包含的属性，使用该对象字面量进行对象赋值、传参都会报错。**

解决这个问题的方法很简单，除了上边的将对象字面量赋值给另一个变量，还可以：

1. 使用类型断言：

In [10]:
let square2 = createSquare({
    color: 'black', 
    width: 10, 
    radius: 5,
} as SquareConfig)

console.log(square2)

{ color: 'black', area: 100 }


undefined

2. 但是如果确定该对象会有更多额外的属性，更好的方法是使用**字符串索引签名**：

In [11]:
interface SquareConfig {
    color?: string;
    width: number;
    [propName: string]: any;
}
let square3 = createSquare({
    color: 'black', 
    width: 10, 
    radius: 5,
})

console.log(square3)

{ color: 'black', area: 100 }


undefined

如非必要，尽量不要绕过这些检查，而应该通过修改 interface 定义来修复这些报错。

## 函数类型

接口除了能定义对象形状外，还可以描述函数类型：

In [12]:
interface SearchFunc {
    (source: string, subString: string): boolean
}

let mySearch: SearchFunc
mySearch = function(src: string, sub: string) {
    let result = src.includes(sub)
    return result
}

// 当然指定了函数类型后不一定需要指定参数类型，因为 `SearchFunc` 类型接口以及指定了参数类型。
mySearch = function(src, sub) {
    let result = src.includes(sub)
    return result
}

console.log(mySearch('hello', 'el'))

true


undefined

## 可索引类型

TypeScript 也可以定义能够索引的类型。如上边的字符串索引，以及数组的数字索引。

下边的接口说明：`StringArray` 类型使用数字进行索引时，会返回一个字符串。

In [13]:
interface StringArray {
    [index: number]: string
}

let myArray: StringArray = ['Luke', 'Leia']

undefined

可以有 `string` 和 `number` 两种作为索引签名，**但是 `number` 索引返回的类型必须是 `string` 索引返回类型的子类型。因为 JavaScript 在进行索引时会将 `number` 类型的转换为 `string` 类型的。**

In [14]:
class Animal {
    name: string
}

class Dog extends Animal {
    breed: string
}

// OK
interface Indexable {
    [index: number]: Dog
    [index: string]: Animal
}

undefined

In [15]:
// Not OK
interface NotIndexable {
    [index: number]: Animal
    [index: string]: Dog
}

Error: Line 3, Character 5
    [index: number]: Animal
____^
TS2413: Numeric index type 'Animal' is not assignable to string index type 'Dog'.

字符串索引类型可以描述一个字典类型，但它也同时指定了所有属性的返回类型。如下例，`name` 属性的返回类型不符合 `index` 的返回类型

In [16]:
interface NotNumberDictionary {
    name: string
    [index: string]: number
}

Error: Line 2, Character 5
    name: string
____^
TS2411: Property 'name' of type 'string' is not assignable to string index type 'number'.

可以声明为 `any` 类型的

In [17]:
interface NumberDictionary {
    name: string
    [index: string]: any
}

let nd: NumberDictionary = {
    name: 'Oxford',
    pages: 678,
}

console.log(nd)

{ name: 'Oxford', pages: 678 }


undefined

可以声明 `readonly`

In [18]:
interface ReadonlyStringArray {
    readonly [index: number]: string
}

let myRSA: ReadonlyStringArray = ['Luke', 'Leia']

undefined

## 类类型

实现一个接口。特别是在类似 C# 和 Java 的语言中，接口的一个最常见作用就是明确一个类的形状。TypeScript 中也可以如此：

In [19]:
interface AnimalInterface {
    name: string
    run(distance): void
}

class Doggy implements AnimalInterface {
    name: string
    
    constructor(name: string) {
        this.name = name
    }
    
    run(distance) {
        console.log(`${this.name} running ${distance}`)
    }
}

let dog = new Doggy('Snoopy')

dog.run(100)

Snoopy running 100


undefined

#### 接口只检查类的实例属性。

类有两方面的类型：静态类型与实例类型。静态类型是类本身的属性，如 contructor。所以下例将会报错

In [20]:
interface CantConstructor {
    new (name: string): void
}

class CanI implements CantConstructor {
    constructor(name: string) {}
}

Error: Line 5, Character 7
class CanI implements CantConstructor {
______^
TS2420: Class 'CanI' incorrectly implements interface 'CantConstructor'.
  Type 'CanI' provides no match for the signature 'new (name: string): void'.

相反，应该使用这个接口来描述类本身：

In [21]:
interface CanConstructor {
    new (name: string): void
}

interface CanInterface {
    name: string
    run(distance: number): void
}

class ICanClass implements CanInterface {
    name: string
    constructor(name: string) {
        this.name = name
    }
    
    run(distance: number) {
        console.log(`${this.name} run ${distance}`)
    }
}

let ICan: CanConstructor = ICanClass

let ican: ICanClass = new ICanClass('i can')
ican.run(100)

i can run 100


undefined

## 接口扩展

和 class 类似，接口也可以进行扩展。

In [22]:
interface Shape {
    color: string
}

interface Circle extends Shape {
    radius: number
}

let circle: Circle = {
    color: 'white',
    radius: 100,
}

// 这里可以使用类型断言：

let circle2 = {} as Circle
circle2.color = 'black'
circle2.radius = 200
console.log(circle2)

{ color: 'black', radius: 200 }


undefined

#### 接口可以扩展多个接口

In [23]:
interface PenStroke {
    penWidth: number
}

interface MyCircle extends Shape, PenStroke {
    radius: number
}

let myCircle: MyCircle = {
    radius: 100,
    color: 'white',
    penWidth: 5,
}

console.log(myCircle)

{ radius: 100, color: 'white', penWidth: 5 }


undefined

#### 一个类也可以实现多个接口

In [24]:
class MC implements Circle, PenStroke {
    color: 'white'
    radius: number
    penWidth: 5
}

undefined

## 混合类型

因为 JavaScript 的动态特性，有时候一个对象会有几种类型。比如下例中的对象既是一个函数，又是对象。这时候就可能需要类型断言等手段来明确描述其类型。

In [25]:
interface Counter {
    (start: number): string
    interval: number
    reset(): void
}

let counter = <Counter>function(start: number) {}
counter.interval = 1
counter.reset = function() {}

console.log(counter)

{ [Function: counter] interval: 1, reset: [Function] }


undefined

## 接口扩展类

当一个接口扩展一个类，接口将继承类的属性，但是不继承类属性的实现。

In [26]:
class Control {
    state: any
}

interface SelectableControl extends Control {
    select(): void;
}

class Button implements SelectableControl {
    state: any;
    select() { }
}

let button: Button = new Button()
button.select()

undefined